In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, hashlib
import numpy as np
from time import time
from pprint import pprint
from typing import Tuple, Dict, Any
import os, hashlib
from qgear.toolbox.Util_H5io4 import write4_data_hdf5

In [ ]:
#| export
def show_CX_block():
    "Display a CX-block circuit with parameters θ and φ"
    from qiskit import QuantumCircuit
    from qiskit.circuit import Parameter
    theta = Parameter('θ')
    phi = Parameter('φ')
    qc = QuantumCircuit(2)
    qc.barrier()
    qc.ry(theta, 0)
    qc.rz(phi, 1)
    qc.cx(0, 1)
    qc.barrier()
    print(qc.draw())

In [ ]:
#| export
def random_qubit_pairs(nq:int, k:int):
    "Generate k random ordered qubit pairs from nq qubits"
    all_pairs = np.array([(i, j) for i in range(nq) for j in range(nq) if i != j])
    selected_indices = np.random.choice(len(all_pairs), k, replace=True)
    return all_pairs[selected_indices]

In [ ]:
#| export
def generate_random_gateList(numQubits:int, numCX:int, numCirc:int):
    "Generate random CX-block circuits and return data, metadata"
    nGate = 3 * numCX
    m = {'h': 1, 'ry': 2, 'rz': 3, 'cx': 4, 'measure': 5}
    circ_type = np.zeros((numCirc, 2), dtype=np.int32)
    gate_type = np.zeros((numCirc, nGate, 3), dtype=np.int32)
    gate_param = np.random.uniform(0, np.pi, size=(numCirc, nGate)).astype(np.float32)
    t_ry = np.full((numCX, 1), m['ry'])
    t_rz = np.full((numCX, 1), m['rz'])
    t_cx = np.full((numCX, 1), m['cx'])

    for j in range(numCirc):
        qpairs = random_qubit_pairs(numQubits, numCX)
        rpairs = qpairs[:, ::-1]
        circ_type[j] = [numQubits, nGate]
        gate_type[j, 0::3] = np.concatenate((t_ry, qpairs), axis=1)
        gate_type[j, 1::3] = np.concatenate((t_rz, rpairs), axis=1)
        gate_type[j, 2::3] = np.concatenate((t_cx, qpairs), axis=1)
        gate_param[j, 2::3] = 0

    outD = {'circ_type': circ_type, 'gate_type': gate_type, 'gate_param': gate_param}
    md = {'gate_map': m, 'num_cx': numCX, 'num_qubit': numQubits,
          'num_gate': nGate, 'num_circ': numCirc}
    return outD, md

In [ ]:
#| export
def save_gateList(outD:Dict[str,Any], md:Dict[str,Any], outPath:str, expName:str=None):
    "Save generated gate list to HDF5 file"
    from Util_H5io4 import write4_data_hdf5
    md['hash'] = hashlib.md5(os.urandom(32)).hexdigest()[:6]
    md['short_name'] = expName if expName else f"rcirc_{md['hash']}"
    outF = os.path.join(outPath, f"{md['short_name']}.gate_list.h5")
    write4_data_hdf5(outD, outF, md)
    pprint(md)
    return outF

In [ ]:
#| export
def get_gpu_info(verb=1):
    import pynvml
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    
    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)

        # Get PCI bus ID
        pci_bus_id_raw = pynvml.nvmlDeviceGetPciInfo(handle).busId
        pci_bus_id = pci_bus_id_raw.decode('utf-8') if isinstance(pci_bus_id_raw, bytes) else pci_bus_id_raw

        # Get GPU model name
        model_name_raw = pynvml.nvmlDeviceGetName(handle)
        model_name = model_name_raw.decode('utf-8') if isinstance(model_name_raw, bytes) else model_name_raw

        # Get memory info
        memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        total_memory = memory_info.total / (1024 ** 2)  # MB
        used_memory = memory_info.used / (1024 ** 2)    # MB
        free_memory = memory_info.free / (1024 ** 2)    # MB

        memGB = float('%.1f' % (total_memory / 1000.))

        if i == 0:
            md = {'gpu_model': model_name, 'tot_mem_gb': memGB, 'pci_bus': [pci_bus_id]}
        else:
            md['pci_bus'].append(pci_bus_id)

        if verb > 1:
            print(f"{i}  {model_name}   bus_id: {pci_bus_id}")
            print(f"Total Memory: {total_memory:.2f} MB")
            print(f"Used Memory: {used_memory:.2f} MB")
            print(f"Free Memory: {free_memory:.2f} MB")
            print()
    
    md['device_count'] = device_count
    pynvml.nvmlShutdown()
    return md